# 1. オペレーティングシステム 演習 14
#  mmapの課題

名前と学生証番号を書け. Enter your name and student ID.

 * 名前 Name:
 * 学生証番号 Student ID:

# 2. mmapを利用した高速な検索
* 整列された配列の2分探索をするbsearchというライブラリ関数がある(man bsearch参照)
* 以下は使用例 (配列aが昇順に整列されていることに注意)

In [ ]:
%%writefile bsearch.c
#com 1
#include <stdio.h>
#include <stdlib.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int key = (argc > 1 ? atoi(argv[1]) : 100);
  int a[] = { 100, 200, 300, 400, 500 };
  int n = sizeof(a) / sizeof(a[0]);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  return 0;
}

In [ ]:
gcc -o bsearch -O3 -Wall bsearch.c

* 実行例

In [ ]:
./bsearch 100
./bsearch 200
./bsearch 400
./bsearch 390

* 以下で, 多数の32 bit整数が隙間なく, 256M個, 昇順に整列されたファイルを作る (512MB)

In [ ]:
#!/usr/bin/python3
import os
import sys
import getpass
import numpy as np

def mk_sorted_file(filename, size):
    assert(size <= 1024 * 1024 * 1024), "don't make it > 4GB"
    rg = np.random.RandomState()
    rg.seed(1234)
    gap = 1024 * 1024 * 1024 / size
    a = rg.randint(0, gap, size=size, dtype=np.uint32)
    s = 0
    for i in range(size):
        s += a[i]
        a[i] = s
    dirname = os.path.dirname(filename)
    if dirname == "":
        dirname = "."
    os.makedirs(dirname, exist_ok=True)
    with open(filename, "wb") as wp:
        a.tofile(wp)

def main():
    filename = sys.argv[1]
    sz = int(sys.argv[2])
    mk_sorted_file(filename, sz)
        
if sys.argv[0].endswith("mk_sorted_file.py"):
    main()


* 多少(20秒くらい)かかるが気長に待つ

In [ ]:
n = 256 * 1024 * 1024
mk_sorted_file("sorted.bin", n)

* 確認

In [ ]:
ls -l sorted.bin

# <font color="green"> Problem 1 :  mmapの有効な利用</font>

* ファイル名と検索したい整数Kを与えられ, その整数がファイル中にあれば,
```
193659 found at 25850-th element
```
のように, その数(193659)がファイル中の何番目(最初の数を0-th elementとする)であるかを表示し, なければ,
```
19365 not found
```
のように, ないと答えるプログラムを, mmapをbsearchを有効に用いて書け
* マニュアルページなどを参照しながら必要な ヘッダファイルを #include して, 一から書いてみよ

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile mmap_bsearch.c
#com 6

#ifpy VER in [1, 3, 5]
int main() {



  return 0;
}
#endifpy
#ifpy VER == 2
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = mmap(0, sz, PROT_READ, MAP_PRIVATE, fd, 0);
  if (a == MAP_FAILED) err(1, "mmap");
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  if (munmap(a, sz) == -1) err(1, "mumap");
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 4
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = malloc(sz);
  if (!a) err(1, "malloc");
  read_sz(fd, a, sz);
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  free(a);
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 6
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  const long n = sz / sizeof(int);
  const long buf_sz = 1024;
  int a[buf_sz];
  int done = 0;
  for (long i = 0; i < n && !done; i += buf_sz) {
    long m = (buf_sz < n - i ? buf_sz : n - i);
    read_sz(fd, a, m * sizeof(int));
    for (long j = 0; j < m; j++) {
      if (a[j] == key) {
        done = 1;
        printf("%d found at %ld-th element\n", key, i + j);
        break;
      } else if (a[j] > key) {
        done = 1;
        printf("%d not found\n", key);
        break;
      }
    }
  }
  if (!done) {
    printf("%d not found\n", key);
  }
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy

In [ ]:
gcc -o mmap_bsearch -Wall -O3 mmap_bsearch.c

* テスト

In [ ]:
data=sorted.bin
if /usr/bin/time ./mmap_bsearch ${data} 362382304 | grep "362382304 found at 241618944-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 322162113 | grep "322162113 found at 214802432-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 281936687 | grep "281936687 found at 187985920-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 241713801 | grep "241713801 found at 161169408-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 362382307 | grep "362382307 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 322162115 | grep "322162115 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 281936688 | grep "281936688 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch ${data} 241713802 | grep "241713802 not found" ; then echo OK ; else echo NG ; fi

In [ ]:
%%writefile mmap_bsearch_ans.c
#com 6

#ifpy VER in [1, 3, 5]
int main() {



  return 0;
}
#endifpy
#ifpy VER == 2
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = mmap(0, sz, PROT_READ, MAP_PRIVATE, fd, 0);
  if (a == MAP_FAILED) err(1, "mmap");
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  if (munmap(a, sz) == -1) err(1, "mumap");
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 4
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = malloc(sz);
  if (!a) err(1, "malloc");
  read_sz(fd, a, sz);
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  free(a);
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 6
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  const long n = sz / sizeof(int);
  const long buf_sz = 1024;
  int a[buf_sz];
  int done = 0;
  for (long i = 0; i < n && !done; i += buf_sz) {
    long m = (buf_sz < n - i ? buf_sz : n - i);
    read_sz(fd, a, m * sizeof(int));
    for (long j = 0; j < m; j++) {
      if (a[j] == key) {
        done = 1;
        printf("%d found at %ld-th element\n", key, i + j);
        break;
      } else if (a[j] > key) {
        done = 1;
        printf("%d not found\n", key);
        break;
      }
    }
  }
  if (!done) {
    printf("%d not found\n", key);
  }
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy

In [ ]:
gcc -o mmap_bsearch_ans -Wall -O3 mmap_bsearch_ans.c

In [ ]:
data=sorted.bin
if /usr/bin/time ./mmap_bsearch_ans ${data} 362382304 | grep "362382304 found at 241618944-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch_ans ${data} 322162113 | grep "322162113 found at 214802432-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch_ans ${data} 281936687 | grep "281936687 found at 187985920-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch_ans ${data} 241713801 | grep "241713801 found at 161169408-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch_ans ${data} 362382307 | grep "362382307 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch_ans ${data} 322162115 | grep "322162115 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch_ans ${data} 281936688 | grep "281936688 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./mmap_bsearch_ans ${data} 241713802 | grep "241713802 not found" ; then echo OK ; else echo NG ; fi

# <font color="green"> Problem 2 :  readで2分探索</font>

同じ仕様のプログラムを, 「read関数でファイルを大きな配列に全て読み込み, その配列をbsearchで探索する」という方法でも実装せよ

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile read_bsearch.c
#com 6

#ifpy VER in [1, 3, 5]
int main() {



  return 0;
}
#endifpy
#ifpy VER == 2
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = mmap(0, sz, PROT_READ, MAP_PRIVATE, fd, 0);
  if (a == MAP_FAILED) err(1, "mmap");
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  if (munmap(a, sz) == -1) err(1, "mumap");
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 4
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = malloc(sz);
  if (!a) err(1, "malloc");
  read_sz(fd, a, sz);
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  free(a);
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 6
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  const long n = sz / sizeof(int);
  const long buf_sz = 1024;
  int a[buf_sz];
  int done = 0;
  for (long i = 0; i < n && !done; i += buf_sz) {
    long m = (buf_sz < n - i ? buf_sz : n - i);
    read_sz(fd, a, m * sizeof(int));
    for (long j = 0; j < m; j++) {
      if (a[j] == key) {
        done = 1;
        printf("%d found at %ld-th element\n", key, i + j);
        break;
      } else if (a[j] > key) {
        done = 1;
        printf("%d not found\n", key);
        break;
      }
    }
  }
  if (!done) {
    printf("%d not found\n", key);
  }
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy

In [ ]:
gcc -o read_bsearch -Wall -O3 read_bsearch.c

In [ ]:
data=sorted.bin
if /usr/bin/time ./read_bsearch ${data} 362382304 | grep "362382304 found at 241618944-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 322162113 | grep "322162113 found at 214802432-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 281936687 | grep "281936687 found at 187985920-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 241713801 | grep "241713801 found at 161169408-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 362382307 | grep "362382307 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 322162115 | grep "322162115 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 281936688 | grep "281936688 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch ${data} 241713802 | grep "241713802 not found" ; then echo OK ; else echo NG ; fi

In [ ]:
%%writefile read_bsearch_ans.c
#com 6

#ifpy VER in [1, 3, 5]
int main() {



  return 0;
}
#endifpy
#ifpy VER == 2
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = mmap(0, sz, PROT_READ, MAP_PRIVATE, fd, 0);
  if (a == MAP_FAILED) err(1, "mmap");
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  if (munmap(a, sz) == -1) err(1, "mumap");
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 4
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = malloc(sz);
  if (!a) err(1, "malloc");
  read_sz(fd, a, sz);
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  free(a);
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 6
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  const long n = sz / sizeof(int);
  const long buf_sz = 1024;
  int a[buf_sz];
  int done = 0;
  for (long i = 0; i < n && !done; i += buf_sz) {
    long m = (buf_sz < n - i ? buf_sz : n - i);
    read_sz(fd, a, m * sizeof(int));
    for (long j = 0; j < m; j++) {
      if (a[j] == key) {
        done = 1;
        printf("%d found at %ld-th element\n", key, i + j);
        break;
      } else if (a[j] > key) {
        done = 1;
        printf("%d not found\n", key);
        break;
      }
    }
  }
  if (!done) {
    printf("%d not found\n", key);
  }
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy

In [ ]:
gcc -o read_bsearch_ans -Wall -O3 read_bsearch_ans.c

In [ ]:
data=sorted.bin
if /usr/bin/time ./read_bsearch_ans ${data} 362382304 | grep "362382304 found at 241618944-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch_ans ${data} 322162113 | grep "322162113 found at 214802432-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch_ans ${data} 281936687 | grep "281936687 found at 187985920-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch_ans ${data} 241713801 | grep "241713801 found at 161169408-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch_ans ${data} 362382307 | grep "362382307 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch_ans ${data} 322162115 | grep "322162115 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch_ans ${data} 281936688 | grep "281936688 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./read_bsearch_ans ${data} 241713802 | grep "241713802 not found" ; then echo OK ; else echo NG ; fi

# <font color="green"> Problem 3 :  線形探索</font>
同じ仕様のプログラムを, 「read関数でファイルを先頭から一定数読んでは, 与えられた値と比較する」ことでも実装せよ

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile lin_search.c
#com 6

#ifpy VER in [1, 3, 5]
int main() {



  return 0;
}
#endifpy
#ifpy VER == 2
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = mmap(0, sz, PROT_READ, MAP_PRIVATE, fd, 0);
  if (a == MAP_FAILED) err(1, "mmap");
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  if (munmap(a, sz) == -1) err(1, "mumap");
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 4
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = malloc(sz);
  if (!a) err(1, "malloc");
  read_sz(fd, a, sz);
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  free(a);
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 6
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  const long n = sz / sizeof(int);
  const long buf_sz = 1024;
  int a[buf_sz];
  int done = 0;
  for (long i = 0; i < n && !done; i += buf_sz) {
    long m = (buf_sz < n - i ? buf_sz : n - i);
    read_sz(fd, a, m * sizeof(int));
    for (long j = 0; j < m; j++) {
      if (a[j] == key) {
        done = 1;
        printf("%d found at %ld-th element\n", key, i + j);
        break;
      } else if (a[j] > key) {
        done = 1;
        printf("%d not found\n", key);
        break;
      }
    }
  }
  if (!done) {
    printf("%d not found\n", key);
  }
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy

In [ ]:
gcc -o lin_search -Wall -O3 lin_search.c

In [ ]:
data=sorted.bin
if /usr/bin/time ./lin_search ${data} 362382304 | grep "362382304 found at 241618944-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search ${data} 322162113 | grep "322162113 found at 214802432-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search ${data} 281936687 | grep "281936687 found at 187985920-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search ${data} 241713801 | grep "241713801 found at 161169408-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search ${data} 362382307 | grep "362382307 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search ${data} 322162115 | grep "322162115 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search ${data} 281936688 | grep "281936688 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search ${data} 241713802 | grep "241713802 not found" ; then echo OK ; else echo NG ; fi

In [ ]:
%%writefile lin_search_ans.c
#com 6

#ifpy VER in [1, 3, 5]
int main() {



  return 0;
}
#endifpy
#ifpy VER == 2
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = mmap(0, sz, PROT_READ, MAP_PRIVATE, fd, 0);
  if (a == MAP_FAILED) err(1, "mmap");
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  if (munmap(a, sz) == -1) err(1, "mumap");
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 4
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  int * a = malloc(sz);
  if (!a) err(1, "malloc");
  read_sz(fd, a, sz);
  const long n = sz / sizeof(int);
  int * found = bsearch(&key, a, n, sizeof(int), compare_int);
  if (found) {
    printf("%d found at %ld-th element\n", key, found - a);
  } else {
    printf("%d not found\n", key);
  }
  free(a);
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy
#ifpy VER == 6
#include <assert.h>
#include <err.h>
#include <fcntl.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <unistd.h>

int compare_int(const void * a_, const void * b_) {
  const int * a = a_;
  const int * b = b_;
  return *a - *b;
}

/* szバイトぴったり読む. その前にEOFになったらエラー */
void read_sz(int fd, void * buf, size_t sz) {
  size_t rd = 0;
  while (rd < sz) {
    ssize_t x = read(fd, buf + rd, sz - rd);
    if (x == -1) err(1, "read");
    assert(x > 0);
    rd += x;
  }
  assert(rd == sz);
}

int main(int argc, char ** argv) {
  int i = 1;
  char * filename = (argc > i ? argv[i] : "sorted.bin"); i++;
  int key         = (argc > i ? atoi(argv[i]) : 100); i++;
  /* ファイルを開く */
  int fd = open(filename, O_RDONLY);
  if (fd == -1) err(1, "open");
  /* ファイルのサイズ */
  struct stat sb[1];
  if (fstat(fd, sb) == -1) err(1, "fstat");
  /* mmap (読み出しのみ) */
  long sz = sb->st_size;
  const long n = sz / sizeof(int);
  const long buf_sz = 1024;
  int a[buf_sz];
  int done = 0;
  for (long i = 0; i < n && !done; i += buf_sz) {
    long m = (buf_sz < n - i ? buf_sz : n - i);
    read_sz(fd, a, m * sizeof(int));
    for (long j = 0; j < m; j++) {
      if (a[j] == key) {
        done = 1;
        printf("%d found at %ld-th element\n", key, i + j);
        break;
      } else if (a[j] > key) {
        done = 1;
        printf("%d not found\n", key);
        break;
      }
    }
  }
  if (!done) {
    printf("%d not found\n", key);
  }
  if (close(fd) == -1) err(1, "close");
  return 0;
}
#endifpy

In [ ]:
gcc -o lin_search_ans -Wall -O3 lin_search_ans.c

In [ ]:
data=sorted.bin
if /usr/bin/time ./lin_search_ans ${data} 362382304 | grep "362382304 found at 241618944-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search_ans ${data} 322162113 | grep "322162113 found at 214802432-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search_ans ${data} 281936687 | grep "281936687 found at 187985920-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search_ans ${data} 241713801 | grep "241713801 found at 161169408-th element" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search_ans ${data} 362382307 | grep "362382307 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search_ans ${data} 322162115 | grep "322162115 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search_ans ${data} 281936688 | grep "281936688 not found" ; then echo OK ; else echo NG ; fi
if /usr/bin/time ./lin_search_ans ${data} 241713802 | grep "241713802 not found" ; then echo OK ; else echo NG ; fi

BEGIN SOLUTION
END SOLUTION


mmap_bsearchが最も速い. lin_searchとread_bsearchは場合にもよるが平均的にはlin_searchが速い

理由:
* mmap_bsearchはファイルからの読み出し量, (読み出し時間を無視した)探索にかかる計算量ともに O(log N) である.
* read_bsearchはファイルからの読み出し量は常にN, (読み出し時間を無視した)探索にかかる計算量はO(log N)
* lin_bsearchはファイルからの読み出し量は qN, (読み出し量を無視した)探索にかかる計算量は O(qN), ここでcは, 対象となる値がファイルのどの位置にあるかを示す値で 0 <= q < 1

上記からmmap_bsearchがほとんど常に最速なのはわかるだろう. read_bsearchとlin_searchについては, まずlin_searchによる探索(線形探索)にかかる時間に比べてファイルからの読み出し時間が支配的であることに注意する必要がある. したがってqがかなり1に近いという場合を除けばlin_searchの方が速いと予想できる.

なおこの問題に対して, read_bsearch の探索時間が O(log N) であるから lin_search よりも速いと結論している解答が非常に多かった. もちろん lin_search との比較は完全に予想できるものではなく, 読み出し時間に比べて(線形)探索の時間が大きいか無視できない場合は, このような結論もありうるが, 少なくとも read_bsearch がすでに O(N) のデータ読込をしているわけだからそっちの時間のほうが明らかに支配的で, 計算時間がO(log N)だから速いという議論は重要な点を見逃している. lin_search が, (平均的には)少ない読み込みで答えを返せることと, 比較の回数が大きいことの, どちらが勝つかという問題である.

また, 実際に実験をしてみて read_bsearch が速かったと言っている人も多かったが,
* 一度に読み込むサイズを適切に設定する (といっても微調整は必要なく, 数KB〜数MBなどおよそどんな値でもOK)
* 毎回同じバッファに読み込む
ようにすれば, lin_searchの方が速くなるはずである

lin_searchで, 一回のreadシステムコールで整数1個分(4バイト)読み込もうとしている人がかなりいたがそのようにすると, それはシステムコールのオーバーヘッドが大きすぎる. 一度に最低でも数KB読み込むようにすればオーバーヘッドはすぐに許容範囲になる

BEGIN SOLUTION
END SOLUTION



mmap_bsearch が最も少なく, 次に lin_search, 一番多いのが read_bsearch

* mmap_bsearchは実際にアクセスしたページ数(log N か, せいぜいその数倍)のメモリが使われるだけ
* lin_searchはアクセスした要素(qN)分のキャッシュメモリ + ユーザプロセス内の読み込むためのバッファの大きさ(プログラムで調整可能な数字だが, 上述したように数KBもあれば十分)が使われる
* read_bsearchは全N要素分のキャッシュメモリ + ユーザプロセス内の同サイズのバッファのためのメモリが使われる(つまりファイルサイズの2倍)
